In [1]:
import pandas as pd
import matplotlib.pyplot as plt

train=pd.read_csv('complex-master-learn-and-get-your-mark/train.csv')
test=pd.read_csv('complex-master-learn-and-get-your-mark/test.csv')

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

X = train.drop(columns=['Id','FreePass'])
X = X.drop(columns=['AvgTime'])

y = train['FreePass']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [11]:
def clean_data(df, ref=None):
    df = df.copy()

    # numéricas
    for col in ['Age','Food','Drinks']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        if ref is not None:
            df[col] = df[col].fillna(ref[col].median())

    # booleanas
    for col in ['Vinyl','VIP']:
        df[col] = df[col].fillna(False).astype(int)

    # categóricas
    for col in ['PreferedAlbum','TicketInfo','Concert']:
        df[col] = df[col].fillna('Missing')

    # ⚠️ Opinion fuera por ahora
    df = df.drop(columns=['Opinion'])

    return df


In [12]:
X_train_c = clean_data(X_train)
X_val_c   = clean_data(X_val, ref=X_train)
X_test_c  = clean_data(test.drop(columns=['Id']), ref=X_train)



In [13]:
from catboost import CatBoostClassifier

cat_cols = ['PreferedAlbum','TicketInfo','Concert']

model = CatBoostClassifier(
    iterations=500,
    depth=6,
    learning_rate=0.05,
    loss_function='Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100
)

model.fit(
    X_train_c, y_train,
    cat_features=cat_cols,
    eval_set=(X_val_c, y_val)
)


0:	learn: 0.7806476	test: 0.7761858	best: 0.7761858 (0)	total: 100ms	remaining: 50s
100:	learn: 0.8063520	test: 0.8008893	best: 0.8008893 (94)	total: 5.94s	remaining: 23.5s
200:	learn: 0.8130252	test: 0.8043478	best: 0.8063241 (181)	total: 11.6s	remaining: 17.3s
300:	learn: 0.8220465	test: 0.8068182	best: 0.8087945 (281)	total: 18.2s	remaining: 12.1s
400:	learn: 0.8316856	test: 0.7989130	best: 0.8087945 (281)	total: 24.8s	remaining: 6.13s
499:	learn: 0.8377410	test: 0.7974308	best: 0.8087945 (281)	total: 30.7s	remaining: 0us

bestTest = 0.8087944664
bestIteration = 281

Shrink model to first 282 iterations.


In [14]:
y_test_pred = model.predict(X_test_c)
y_test_pred = y_test_pred.flatten()
submission = pd.DataFrame({
    'Id': test['Id'],
    'FreePass': y_test_pred
})

submission.to_csv('submission_catboost.csv', index=False)
print("✅ CSV generado: submission_catboost.csv")


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=2]=1.531 : cat_features must be integer or string, real number values and NaN values should be converted to string.